[View in Colaboratory](https://colab.research.google.com/github/tcglarry/aoi/blob/master/aoi_inceptionv3.ipynb)

In [13]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18408 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8.1) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .

In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [15]:
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [16]:
!mkdir -p drive
!google-drive-ocamlfuse drive
!ls drive

aoi_aoi_test1.ipynb  PiXNet				      test for excel
aoi_test	     Sample upload.txt			      Untitled0.ipynb
Capstone_PixNet.zip  TAHU PowerPoint Template.pdf	      Untitled1.ipynb
Colab Notebooks      TAHU PowerPoint Template - 柱狀圖 1.ods  Untitled2.ipynb


In [0]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image

from keras.layers import Dense, GlobalAveragePooling2D


import cv2
import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf
import subprocess
import os
import pickle
from keras.layers import Input, Conv2D, MaxPooling2D, merge, Lambda,UpSampling2D, concatenate
from keras.models import Model
import pandas as pd
import sklearn 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split


In [0]:
src = 'drive/aoi_test/'

In [0]:
train_img = os.listdir(src+'train_images')
print ('iamge lies size', len(train_img))
img_list=[]
for i,img in enumerate(train_img):
    x = cv2.imread(src+'train_images/'+img)
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    #x = x/127.5 -1.
    img_list.append(x)
    
print ('done')

iamge lies size 2528

In [0]:
train_data = np.array(img_list)
print (train_data.shape)
#train_data =  train_data[:,:,:,np.newaxis]
print (train_data.shape)

(2528, 512, 512, 3)
(2528, 512, 512, 3)


In [0]:
'''
with open(src+'train_data.pickle', 'wb') as handle:
    pickle.dump(train_data, handle)
'''

In [6]:
print ('here')

aoi_aoi_test1.ipynb  PiXNet				      test for excel
aoi_test	     Sample upload.txt			      Untitled0.ipynb
Capstone_PixNet.zip  TAHU PowerPoint Template.pdf
Colab Notebooks      TAHU PowerPoint Template - 柱狀圖 1.ods


In [1]:
df_y = pd.read_csv(src+'train.csv')
print (df_y['Label'].value_counts())
train_y =  df_y['Label'].values

onehot_encoder = OneHotEncoder(sparse=False)
train_y = train_y.reshape(len(train_y), 1)
print (train_y.shape)
train_label = onehot_encoder.fit_transform(train_y)


print ('beofre', train_label.shape)
print (df_y['Label'][:10])
print (train_label[:10,:])

train_label = train_label[:,np.newaxis,np.newaxis,:]

print ('after', train_label.shape)


NameError: ignored

In [23]:
inputs = Input(shape=(512,512,3))
inputs = Lambda(lambda x: x/275. - 1.0)(inputs)
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet',input_tensor=inputs, input_shape=(512,512,3), include_top=False)
for layer in base_model.layers:
    layer.trainable = False

87916544/87910968 [==============================] - 3s 0us/step


In [24]:
# add a global spatial average pooling layer
x = base_model.output
print ('shape of inception output', x.get_shape())
x = MaxPooling2D(pool_size=(2,2))(x)

#x = GlobalAveragePooling2D()(x)
print ('shape of pooling output', x.get_shape())
# let's add a fully-connected layer
x = Conv2D(32,(5,5), activation='relu', padding='valid')(x)
x = Conv2D(128,(3,3), activation='relu', padding='valid')(x)

x = Dense(256, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(6, activation='softmax')(x)

shape of inception output (?, 14, 14, 2048)
shape of pooling output (?, 7, 7, 2048)


In [25]:
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 512, 512, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 255, 255, 32) 864         lambda_2[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 255, 255, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation

In [0]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False


In [0]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])


In [0]:
# train the model on the new data for a few epochs

for i in range (100):
  if os.path.isfile(src+'aoi_inceptionv3.h5'):
    
    model = load_model(src+'aoi_inceptionv3.h5')
    print ('model laoded')
  else:
    print ('model not exist')
  train_x,val_x,train_y,val_y = train_test_split(train_data,train_label,test_size=0.2, shuffle=True)
  
  model.fit(x=train_x, y=train_y, batch_size=32, epochs=100, verbose=1, validation_split=0.2,  shuffle=True)
  model.save(src+'aoi_inceptionv3.h5')







model not exist
Train on 1617 samples, validate on 405 samples
Epoch 1/100
 448/1617 [=======>......................] - ETA: 45:10 - loss: 10.9305

In [0]:
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

In [0]:
# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

In [0]:
# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True


In [0]:
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')


In [0]:
# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit_generator(...)